In [7]:
import cv2

if hasattr(cv2, "aruco"):
    print("Aruco is available.")


Aruco is available.


In [8]:
import cv2
import numpy as np

# Open video
cap = cv2.VideoCapture('aruco.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Setup video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('aruco_green_overlay.mp4', fourcc, fps, (frame_w, frame_h))

# Setup ArUco detector with latest API
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

# Expected marker IDs at corners (from your booth/cv.py)
expected_ids = [0, 1, 2, 3]
margin = 10  # The margin you defined in generate_aruco_board_img
marker_size = 80  # The marker size you defined

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Create a copy for overlay
    overlay = frame.copy()
    
    # Detect ArUco markers
    corners, ids, _ = detector.detectMarkers(frame)
    
    if ids is not None and len(ids) >= 4:
        # Check if we have all 4 expected markers
        detected_markers = {}
        
        for i, marker_id in enumerate(ids.flatten()):
            if marker_id in expected_ids:
                # Get the center of the marker
                marker_corners = corners[i][0]
                center = np.mean(marker_corners, axis=0)
                detected_markers[marker_id] = center
        
        # If we have all 4 markers, create the screen boundary
        if len(detected_markers) == 4:
            # Get marker centers in order: [TL, TR, BR, BL]
            marker_centers = []
            for marker_id in expected_ids:
                if marker_id in detected_markers:
                    center = detected_markers[marker_id]
                    marker_centers.append(center)
            
            if len(marker_centers) == 4:
                # Convert to numpy array
                marker_centers = np.array(marker_centers, dtype=np.float32)
                
                # The markers are at the corners of your board, but we want to extend
                # outward by the margin to cover the full screen area
                # Marker layout: [TL, TR, BR, BL] based on your booth/cv.py
                tl_marker = marker_centers[0]  # Top-left marker center
                tr_marker = marker_centers[1]  # Top-right marker center  
                br_marker = marker_centers[2]  # Bottom-right marker center
                bl_marker = marker_centers[3]  # Bottom-left marker center
                
                # Extend outward from marker centers to get screen corners
                # The markers are positioned at (margin, margin) etc. from screen edges
                # So we extend by (margin + marker_size/2) to get to screen edge
                extend_distance = margin + marker_size // 2
                
                # Calculate screen corners by extending from marker centers
                tl_screen = tl_marker - [extend_distance, extend_distance]
                tr_screen = tr_marker + [extend_distance, -extend_distance]
                br_screen = br_marker + [extend_distance, extend_distance]
                bl_screen = bl_marker + [-extend_distance, extend_distance]
                
                screen_corners = np.array([
                    tl_screen, tr_screen, br_screen, bl_screen
                ], dtype=np.int32)
                
                # Create green overlay
                green_overlay = np.zeros_like(frame)
                cv2.fillPoly(green_overlay, [screen_corners], (0, 255, 0))
                
                # Blend with original frame
                alpha = 0.3  # Transparency level
                overlay = cv2.addWeighted(frame, 1 - alpha, green_overlay, alpha, 0)
                
                # Draw screen boundary for visualization
                cv2.polylines(overlay, [screen_corners], True, (0, 255, 0), 3)
                
                # Optionally draw marker centers for debugging
                for i, center in enumerate(marker_centers):
                    cv2.circle(overlay, tuple(center.astype(int)), 5, (255, 0, 0), -1)
                    cv2.putText(overlay, str(expected_ids[i]), 
                              tuple((center + [10, 10]).astype(int)), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    # Write frame
    out.write(overlay)

# Clean up
cap.release()
out.release()
print("Video processing complete! Output saved as 'aruco_green_overlay.mp4'")

Video processing complete! Output saved as 'aruco_green_overlay.mp4'
